# Set up

In [2]:
import getpass
import pandas as pd
import numpy as np
# import matplotlib as mplot
import urllib.parse
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import Markdown as md
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import pandas as pd
import plotly.express as px
import pytz

pd.set_option('display.max_columns', None)

**Connection**

Enter the EDM server address and the login credentials provided by Awesense. If you do not have the credentials, or have any trouble connecting, please contact api@awesense.com.
<span style='color:red'> **Please do NOT store the credentials in the notebook, nor share them with anyone.** </span>

In [3]:
# Input creditials by code (need to remove your private info before you pushed it)
edm_address = ""
edm_name = ""
edm_password = ""

# User input the creditials manually 
edm_address = getpass.getpass(prompt='EDM server address: ')

print('\nEDM login information')
edm_name = getpass.getpass(prompt='Username: ')
edm_password = getpass.getpass(prompt='Password: ')

edm_password = urllib.parse.quote(edm_password)

%load_ext sql
%sql postgresql://$edm_name:$edm_password@$edm_address/edm
%config SqlMagic.displaycon = False
%config SqlMagic.feedback = False

# Delete the credential variables for security purposes.

del edm_name, edm_password

#### Input Parameters
Input the grid name to find all the high-level transformers in this grid. 

In [4]:
# User input for the grid.
# grid_id = input('Enter grid ID: ') # awefice

# For simplitiy, set grid_id = awefice by code
grid_id = 'North Central Zone'

In [5]:
result = %sql SELECT grid_element_id, \
                        is_producer,\
                        meta, \
                        phases\
                FROM grid_element\
                WHERE grid_id = '{grid_id}'\
                    AND is_producer = True\
                    AND type = 'Transformer';

# Convert the results to a data frame.
df_transformers = result.DataFrame()

# Pull out the information from `meta` column saved as JSONB.
df_transformers = pd.concat([df_transformers.drop(['meta'], axis=1),
                             df_transformers['meta'].apply(pd.Series)], axis=1)

print(df_transformers.columns)

# Choose the relevant columns to display transformers. 
df_transformers = df_transformers[['grid_element_id', 'is_producer', 'ownership', 'rating_kva', 
                                   'phases', 'voltage_level', 'hvmv_parent_element', 
                                   'primary_voltage', 'secondary_voltage']]

# Display the results.
df_transformers

Index(['grid_element_id', 'is_producer', 'phases', 'name', 'label', 'latitude',
       'load_loss', 'longitude', 'mv_feeder', 'ownership', 'make_model',
       'rating_kva', 'enclosure_id', 'no_load_loss', 'voltage_level',
       'commission_date', 'primary_voltage', 'maintenance_type',
       'manufacture_date', 'installation_type', 'secondary_voltage',
       'hvmv_parent_element', 'is_operation_status', 'parent_transformer_id',
       'winding_configuration'],
      dtype='object')


,grid_element_id,is_producer,ownership,rating_kva,phases,voltage_level,hvmv_parent_element,primary_voltage,secondary_voltage
0,121175_hvmv,True,None,500.0,ABC,HV/MV,None,200000,14000
1,12465_hvmv,True,None,1800.0,ABC,HV/MV,None,200000,14000
2,48504_hvmv,True,None,700.0,ABC,HV/MV,None,200000,14000
3,10407_hvmv,True,None,3100.0,ABC,HV/MV,None,200000,14000
4,28538_hvmv,True,None,2200.0,ABC,HV/MV,None,200000,14000
5,10680_hvmv,True,None,2300.0,ABC,HV/MV,None,200000,14000
6,10954_hvmv,True,None,2200.0,ABC,HV/MV,None,200000,14000
7,20593_hvmv,True,None,4000.0,ABC,HV/MV,None,200000,14000
8,21499_hvmv,True,None,1100.0,ABC,HV/MV,None,200000,14000
9,22941_hvmv,True,None,1700.0,ABC,HV/MV,None,200000,14000


---

### Define the grid id and grid element that you would like to investigate

In [6]:
grid_id = 'North Central Zone'
grid_element_id = '12373_hvmv'

### Create View grid_element_metric to grab meters and other grid elements info

In [7]:
%%sql
    
CREATE OR REPLACE TEMPORARY VIEW grid_element_metric AS
    SELECT grid_id,
            grid_element_id,
            phases,
            type,
            provider,
            direction,
            friendly_id,
            metric_key AS metric,
            valid,
            timestamp,
            value
    FROM grid_element_data_source geds
    JOIN UNNEST(geds.metrics::TEXT[]) AS metric_key
        ON true
    LEFT JOIN ts_data_source_select(grid_element_data_source_id, metric_key) AS ts
        ON true;

[]

### Create View meter_data_source

In [8]:
%%sql

CREATE OR REPLACE TEMPORARY VIEW meter_data_source2 AS
    SELECT meter.grid_id,
            meter.grid_element_id,
            geds.grid_element_data_source_id,
            geds.friendly_id,
            geds.phases,
            geds.provider,
            metric_key as metric,
            lower(geds.valid) as start_time,
            upper(geds.valid) as end_time
    FROM grid_get_downstream('{grid_id}', '{grid_element_id}') AS meter
    LEFT JOIN grid_element_data_source geds
        ON meter.grid_element_id = geds.grid_element_id
        AND meter.grid_id = geds.grid_id
        AND geds.type = 'CONSUMER'
    JOIN UNNEST(geds.metrics::TEXT[]) AS metric_key
        ON true
    WHERE meter.type = 'Meter';

[]

### Create View meter_consumption

In [9]:
%%sql

CREATE OR REPLACE TEMPORARY VIEW meter_consumption2 AS
SELECT meter.grid_id,
        meter.grid_element_id,
        meter.friendly_id,
        meter.phases,
        timestamp,
        value AS kWh
FROM meter_data_source2 meter
LEFT JOIN grid_element_metric gem
    ON gem.grid_id = meter.grid_id
    AND gem.grid_element_id = meter.grid_element_id
WHERE gem.metric = 'kWh'
   AND gem.type = 'CONSUMER';

[]

In [10]:
%%sql

WITH ts_stats AS (
    SELECT SUM(kWh) AS kWh, MIN(timestamp) AS start_timerange, MAX(timestamp) AS end_timerange
    FROM meter_consumption2
)
SELECT name, value FROM (
    SELECT 1 AS idx, 'Meters Found' AS name, (SELECT COUNT(DISTINCT grid_element_id) FROM meter_data_source2)::text AS value
    UNION
    SELECT 2, 'Meters w/ Datasources', (SELECT COUNT(DISTINCT grid_element_id) FROM meter_data_source2 WHERE grid_element_data_source_id IS NOT NULL)::text
    UNION
    SELECT 3, 'Common DS Timerange', (SELECT CONCAT(MAX(start_time), ' - ',  MIN(end_time)) FROM meter_data_source2)::text
    UNION
    SELECT 4, 'Common Timeseries Timerange', (SELECT CONCAT(start_timerange, ' - ', end_timerange) FROM ts_stats)::text
    UNION
    SELECT 5, 'Total Consumption', (SELECT kwh FROM ts_stats)::text
) x ORDER BY idx
;

name,value
Meters Found,316
Meters w/ Datasources,316
Common DS Timerange,2021-01-01 08:00:00+00 -
Common Timeseries Timerange,2021-01-01 05:00:00+00 - 2024-06-18 19:00:00+00
Total Consumption,33250481.38449925


In [11]:
# grab all information from sql and save to a python varable
# the information is restricted to timestamp >= '2024-01-01 00:00:00+00:00'
meter_consumption = %sql SELECT grid_id,\
                grid_element_id,\
                friendly_id,\
                phases,\
                timestamp,\
                kWh\
            FROM meter_consumption2\
            WHERE timestamp >= '2024-01-01 00:00:00+00:00';
                    
# Sort the data by date saved as `month`.
meter_consumption_df = meter_consumption.DataFrame()

In [12]:
meter_consumption_df.head()

,grid_id,grid_element_id,friendly_id,phases,timestamp,kwh
0,North Central Zone,17AY10-1,17AY10-1,B,2024-01-01 00:00:00+00:00,1.366261
1,North Central Zone,17AY10-1,17AY10-1,B,2024-01-01 01:00:00+00:00,2.120388
2,North Central Zone,17AY10-1,17AY10-1,B,2024-01-01 02:00:00+00:00,1.567344
3,North Central Zone,17AY10-1,17AY10-1,B,2024-01-01 03:00:00+00:00,1.822171
4,North Central Zone,17AY10-1,17AY10-1,B,2024-01-01 04:00:00+00:00,1.434942


In [13]:
# Create columns to indicate the month, day and hour information
meter_consumption_df['month'] = meter_consumption_df['timestamp'].dt.to_period('M').dt.to_timestamp().dt.date
meter_consumption_df['day'] = meter_consumption_df['timestamp'].dt.to_period('D').dt.to_timestamp().dt.date
meter_consumption_df['hour'] = meter_consumption_df['timestamp']

/var/folders/8f/xm08m3255q9b0j33fhp73q_w0000gn/T/ipykernel_14140/2765522095.py:2: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  meter_consumption_df['month'] = meter_consumption_df['timestamp'].dt.to_period('M').dt.to_timestamp().dt.date
/var/folders/8f/xm08m3255q9b0j33fhp73q_w0000gn/T/ipykernel_14140/2765522095.py:3: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  meter_consumption_df['day'] = meter_consumption_df['timestamp'].dt.to_period('D').dt.to_timestamp().dt.date


In [14]:
meter_consumption_df

,grid_id,grid_element_id,friendly_id,phases,timestamp,kwh,month,day,hour
0,North Central Zone,17AY10-1,17AY10-1,B,2024-01-01 00:00:00+00:00,1.366261,2024-01-01,2024-01-01,2024-01-01 00:00:00+00:00
1,North Central Zone,17AY10-1,17AY10-1,B,2024-01-01 01:00:00+00:00,2.120388,2024-01-01,2024-01-01,2024-01-01 01:00:00+00:00
2,North Central Zone,17AY10-1,17AY10-1,B,2024-01-01 02:00:00+00:00,1.567344,2024-01-01,2024-01-01,2024-01-01 02:00:00+00:00
3,North Central Zone,17AY10-1,17AY10-1,B,2024-01-01 03:00:00+00:00,1.822171,2024-01-01,2024-01-01,2024-01-01 03:00:00+00:00
4,North Central Zone,17AY10-1,17AY10-1,B,2024-01-01 04:00:00+00:00,1.434942,2024-01-01,2024-01-01,2024-01-01 04:00:00+00:00
...,...,...,...,...,...,...,...,...,...
2576027,North Central Zone,17Y95-1,17Y95-1,A,2024-06-18 15:00:00+00:00,2.847628,2024-06-01,2024-06-18,2024-06-18 15:00:00+00:00
2576028,North Central Zone,17Y95-1,17Y95-1,A,2024-06-18 16:00:00+00:00,2.308548,2024-06-01,2024-06-18,2024-06-18 16:00:00+00:00
2576029,North Central Zone,17Y95-1,17Y95-1,A,2024-06-18 17:00:00+00:00,2.526070,2024-06-01,2024-06-18,2024-06-18 17:00:00+00:00
2576030,North Central Zone,17Y95-1,17Y95-1,A,2024-06-18 18:00:00+00:00,1.950663,2024-06-01,2024-06-18,2024-06-18 18:00:00+00:00


In [15]:
meter_list = meter_consumption_df['grid_element_id'].unique()

In [16]:
element_downstream = %sql SELECT grid_id, grid_element_id, phases, is_producer, terminal1_cn, terminal2_cn FROM grid_get_nearby('{grid_id}', '{grid_element_id}', 200)

In [17]:
element_downstream = element_downstream.DataFrame()
element_downstream

,grid_id,grid_element_id,phases,is_producer,terminal1_cn,terminal2_cn
0,North Central Zone,9496,B,False,VCN.L17-57921,VCN.9496
1,North Central Zone,42574,A,False,VCN.L17-57643,VCN.42574
2,North Central Zone,120972,C,False,VCN.L17-58188,VCN.120972
3,North Central Zone,2553,B,False,VCN.L17-57843,VCN.2553
4,North Central Zone,L17-131416,B,False,VCN.L17-57814,VCN.L17-131416
...,...,...,...,...,...,...
2056,North Central Zone,26758,A,False,VCN.102101,VCN.26758
2057,North Central Zone,14817,A,False,VCN.48566,VCN.14817
2058,North Central Zone,17Y169,A,False,VCN.27011,VCN.17Y169
2059,North Central Zone,L17-58272,B,False,VCN.L17-58271,VCN.L17-58272


In [18]:
# Algorithm for grouping the meters according to its closest junction

meter_terminals = []

for meter in meter_list:
    trace_element = element_downstream[element_downstream['grid_element_id']==meter]
    junction = meter
    terminal = ""
    phase = "ABC"
    
    if len(trace_element['phases'].iloc[0])==1:
        while len(trace_element['phases'].iloc[0])==1:
            terminal = trace_element['terminal1_cn'].iloc[0]
            trace_element = element_downstream[element_downstream['terminal2_cn']==terminal]
        junction = trace_element['grid_element_id'].iloc[0]
    else:
        terminal = trace_element['terminal1_cn'].iloc[0]
        
    meter_terminals.append([meter, junction, terminal])
    
meter_group_df = pd.DataFrame(data=meter_terminals, columns=['grid_element_id', 'junction', 'junction_terminal1'])
meter_group_df


,grid_element_id,junction,junction_terminal1
0,17AY10-1,L17-57718,VCN.L17-57718
1,17AY10-1ZA,L17-57718,VCN.L17-57718
2,17AY13-1,L17-57718,VCN.L17-57718
3,17AY14,L17-57718,VCN.L17-57718
4,17AY14-1,L17-57718,VCN.L17-57718
...,...,...,...
311,17Y91-4-1ZA,L17-58332,VCN.L17-58332
312,17Y91-7,L17-58338,VCN.L17-58338
313,17Y92ZA,L17-58324,VCN.L17-58324
314,17Y92ZB,L17-58324,VCN.L17-58324


In [19]:
len(meter_group_df['junction_terminal1'].unique())

34

In [20]:
# Concatenate meter consumption data with meter groups
meter_consumption_df = meter_consumption_df.merge(meter_group_df, on='grid_element_id', how='left')
meter_consumption_df

,grid_id,grid_element_id,friendly_id,phases,timestamp,kwh,month,day,hour,junction,junction_terminal1
0,North Central Zone,17AY10-1,17AY10-1,B,2024-01-01 00:00:00+00:00,1.366261,2024-01-01,2024-01-01,2024-01-01 00:00:00+00:00,L17-57718,VCN.L17-57718
1,North Central Zone,17AY10-1,17AY10-1,B,2024-01-01 01:00:00+00:00,2.120388,2024-01-01,2024-01-01,2024-01-01 01:00:00+00:00,L17-57718,VCN.L17-57718
2,North Central Zone,17AY10-1,17AY10-1,B,2024-01-01 02:00:00+00:00,1.567344,2024-01-01,2024-01-01,2024-01-01 02:00:00+00:00,L17-57718,VCN.L17-57718
3,North Central Zone,17AY10-1,17AY10-1,B,2024-01-01 03:00:00+00:00,1.822171,2024-01-01,2024-01-01,2024-01-01 03:00:00+00:00,L17-57718,VCN.L17-57718
4,North Central Zone,17AY10-1,17AY10-1,B,2024-01-01 04:00:00+00:00,1.434942,2024-01-01,2024-01-01,2024-01-01 04:00:00+00:00,L17-57718,VCN.L17-57718
...,...,...,...,...,...,...,...,...,...,...,...
2576027,North Central Zone,17Y95-1,17Y95-1,A,2024-06-18 15:00:00+00:00,2.847628,2024-06-01,2024-06-18,2024-06-18 15:00:00+00:00,L17-58324,VCN.L17-58324
2576028,North Central Zone,17Y95-1,17Y95-1,A,2024-06-18 16:00:00+00:00,2.308548,2024-06-01,2024-06-18,2024-06-18 16:00:00+00:00,L17-58324,VCN.L17-58324
2576029,North Central Zone,17Y95-1,17Y95-1,A,2024-06-18 17:00:00+00:00,2.526070,2024-06-01,2024-06-18,2024-06-18 17:00:00+00:00,L17-58324,VCN.L17-58324
2576030,North Central Zone,17Y95-1,17Y95-1,A,2024-06-18 18:00:00+00:00,1.950663,2024-06-01,2024-06-18,2024-06-18 18:00:00+00:00,L17-58324,VCN.L17-58324


# Group the data 

### From this point, we can group data according to your needs and do some filtering if necessary

### Use Case 1: by meter and month

In [21]:
meter_month_df = meter_consumption_df.groupby(['friendly_id', 'month'])['kwh'].mean().reset_index()
meter_month_df

,friendly_id,month,kwh
0,17AY10-1,2024-01-01,2.054317
1,17AY10-1,2024-02-01,2.083195
2,17AY10-1,2024-03-01,1.825234
3,17AY10-1,2024-04-01,1.900568
4,17AY10-1,2024-05-01,2.396772
...,...,...,...
1891,17Y95-1,2024-02-01,1.528928
1892,17Y95-1,2024-03-01,1.354950
1893,17Y95-1,2024-04-01,1.390599
1894,17Y95-1,2024-05-01,1.721543


In [22]:
# Plot graph
px.line(meter_month_df, x='month', y='kwh', 
        title='Average Hourly Consumption in each Month by Meter',
        color='friendly_id')

### Use Case 2: by phase and month

In [23]:
phases_month_df = meter_consumption_df.groupby(['phases', 'month'])['kwh'].mean().reset_index()
phases_month_df

,phases,month,kwh
0,A,2024-01-01,1.543567
1,A,2024-02-01,1.460182
2,A,2024-03-01,1.296972
3,A,2024-04-01,1.327771
4,A,2024-05-01,1.632292
5,A,2024-06-01,2.183106
6,B,2024-01-01,1.516935
7,B,2024-02-01,1.426269
8,B,2024-03-01,1.263031
9,B,2024-04-01,1.306747


In [24]:
# Plot graph
px.line(phases_month_df, x='month', y='kwh', 
        title='Average Hourly Consumption in each Month by Meter',
        color='phases')

### Use Case 3: by phase, cluster and month

In [25]:
cluster_month_df = meter_consumption_df.groupby(['phases', 'junction', 'month'])['kwh'].mean().reset_index()

In [26]:
# Plot graph
px.line(cluster_month_df, x='month', y='kwh', 
        title='Average Hourly Consumption in each Month by Cluster',
        color='phases')